In [1]:
import ee
import datetime
import os
import itertools
import sys

from pprint import pprint
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

import geemap

import subprocess
from subprocess import PIPE

In [2]:
ee.Initialize()

# append the parent path into sys-path so wen can import necessary modules
sys.path.append('../')

In [3]:
from BackGround_modules.Class_1_Make_fourier_imgs import Make_Fourier
from BackGround_modules.Class_2_Classify_Fourier_Img import Classification
from BackGround_modules.Class_3_Calculate_the_accuracy import Accuracy_assesment

The default supplementary data a Zero img, change accordingly when classify other years imgs



#### Step_0_Define_Basic_Parameters

In [4]:
# imprt North_China_Plain boundary
North_China_Plain = ee.FeatureCollection("users/wangjinzhulala/North_China_Plain_Python/Boundary_shp/North_China_Plain_Boundary")

In [5]:
# Define the time-range
# just get one_range from every different sensors
year_range = [(f'{i}-01-01',f'2019-12-31') for i in range(2015,2020)] + \
             [(f'{i}-01-01',f'2013-12-31') for i in range(2009,2014)] + \
             [(f'{i}-01-01',f'2010-12-31') for i in range(2006,2011)]


In [6]:
year_range

[('2015-01-01', '2019-12-31'),
 ('2016-01-01', '2019-12-31'),
 ('2017-01-01', '2019-12-31'),
 ('2018-01-01', '2019-12-31'),
 ('2019-01-01', '2019-12-31'),
 ('2009-01-01', '2013-12-31'),
 ('2010-01-01', '2013-12-31'),
 ('2011-01-01', '2013-12-31'),
 ('2012-01-01', '2013-12-31'),
 ('2013-01-01', '2013-12-31'),
 ('2006-01-01', '2010-12-31'),
 ('2007-01-01', '2010-12-31'),
 ('2008-01-01', '2010-12-31'),
 ('2009-01-01', '2010-12-31'),
 ('2010-01-01', '2010-12-31')]

#### Step_2_Triple loop to create GEE_instaces for later residule computation

In [7]:
# randomly generated 100 point within the boundary of North China Plain
# which will be used to record the residul between original and fitted
# normalized value
point_num = 100
random_point = ee.FeatureCollection.randomPoints(North_China_Plain, point_num, 101).toList(point_num).getInfo()

# define the max harmonic number that will be tested
harmo_num = 10

In [ ]:
# innitialize the list that holds residule vlaue
point_with_residule = {}

# define an operation flag to record the number of operations
op_flag   = 0
op_totall = len(year_range) * harmo_num * point_num

# __________________________________loop through the year_range________________________________
for span in year_range:
    
    # define the start and end of time
    start = span[0]
    end   = span[1]
    spane_len = str(int(end[:4]) - int(start[:4]))
    
    # get span_name for export
    split = '-'
    span_name = f'{start.split(split)[0]}_{end.split(split)[0]}'
    
    # innitilize a 1st level dict to hold residule-value of span_name-[harmonic]
    point_with_residule[span_name] = {}
    
    # __________________________________loop through harmonics ________________________________
    for harmonic in range(1,harmo_num + 1):

        # make a residule img 
        Fourier_making = Make_Fourier(start_date=start,end_date=end,harmonics=harmonic)
        Fourier_making.Stp_1_Create_hamonic_names()
        Fourier_making.Stp_2_Add_harmonics()
        Fourier_making.Stp_3_Harmonic_fit()

        # get the Residule img.
        Residule_img = ee.ImageCollection(Fourier_making.harmonicTrendResidule)

        
        # innitilize the 2ed level dict to hold residule-value of span_name_harmonic-[point_num]
        point_with_residule[span_name][harmonic] = {}
        
        # ______________________________loop through each point _______________________________
        for pt in random_point:
            
            # updata the opration number
            op_flag += 1
            
            # get the id of the point
            pt_id = pt['id']
            
            # calculate the residule, this is ght 3rd level of the dict
            point_with_residule[span_name][harmonic][int(pt_id)] = Residule_img.getRegion(pt['geometry'],30)

            #print out the process out point calculation
            print(f'{span_name}_harmonic_{harmonic}_pt_{pt_id.zfill(2)}. ==>{str(op_flag).zfill(5)}/{op_totall}')

        # print out the process of Harmonic
        print(f'_____Harmonic_{harmonic}____')

        
    # print out the process of year_span
    print(f'===============|{span_name}|==================')

Analyzing the images of 2015_2019
2015_2019_harmonic_1_pt_00. ==>00001/15000
2015_2019_harmonic_1_pt_01. ==>00002/15000
2015_2019_harmonic_1_pt_02. ==>00003/15000
2015_2019_harmonic_1_pt_03. ==>00004/15000
2015_2019_harmonic_1_pt_04. ==>00005/15000
2015_2019_harmonic_1_pt_05. ==>00006/15000
2015_2019_harmonic_1_pt_06. ==>00007/15000
2015_2019_harmonic_1_pt_07. ==>00008/15000
2015_2019_harmonic_1_pt_08. ==>00009/15000
2015_2019_harmonic_1_pt_09. ==>00010/15000
2015_2019_harmonic_1_pt_10. ==>00011/15000
2015_2019_harmonic_1_pt_11. ==>00012/15000
2015_2019_harmonic_1_pt_12. ==>00013/15000
2015_2019_harmonic_1_pt_13. ==>00014/15000
2015_2019_harmonic_1_pt_14. ==>00015/15000
2015_2019_harmonic_1_pt_15. ==>00016/15000
2015_2019_harmonic_1_pt_16. ==>00017/15000
2015_2019_harmonic_1_pt_17. ==>00018/15000
2015_2019_harmonic_1_pt_18. ==>00019/15000
2015_2019_harmonic_1_pt_19. ==>00020/15000
2015_2019_harmonic_1_pt_20. ==>00021/15000
2015_2019_harmonic_1_pt_21. ==>00022/15000
2015_2019_harmonic_1

Analyzing the images of 2015_2019
2015_2019_harmonic_3_pt_00. ==>00201/15000
2015_2019_harmonic_3_pt_01. ==>00202/15000
2015_2019_harmonic_3_pt_02. ==>00203/15000
2015_2019_harmonic_3_pt_03. ==>00204/15000
2015_2019_harmonic_3_pt_04. ==>00205/15000
2015_2019_harmonic_3_pt_05. ==>00206/15000
2015_2019_harmonic_3_pt_06. ==>00207/15000
2015_2019_harmonic_3_pt_07. ==>00208/15000
2015_2019_harmonic_3_pt_08. ==>00209/15000
2015_2019_harmonic_3_pt_09. ==>00210/15000
2015_2019_harmonic_3_pt_10. ==>00211/15000
2015_2019_harmonic_3_pt_11. ==>00212/15000
2015_2019_harmonic_3_pt_12. ==>00213/15000
2015_2019_harmonic_3_pt_13. ==>00214/15000
2015_2019_harmonic_3_pt_14. ==>00215/15000
2015_2019_harmonic_3_pt_15. ==>00216/15000
2015_2019_harmonic_3_pt_16. ==>00217/15000
2015_2019_harmonic_3_pt_17. ==>00218/15000
2015_2019_harmonic_3_pt_18. ==>00219/15000
2015_2019_harmonic_3_pt_19. ==>00220/15000
2015_2019_harmonic_3_pt_20. ==>00221/15000
2015_2019_harmonic_3_pt_21. ==>00222/15000
2015_2019_harmonic_3

#### Step_3_Unpack the residule instances using getInfo

In [ ]:
# unpack the point_with_residule dictionary and fectch 
# only residule information

# set opration flag to report the process
ops_num   = 0
ops_total = len(year_range) * harmo_num


# innitiate the residule dict
Residule = {}

# loop through years
for year in point_with_residule.keys():
    
    # loop through harmonics
    for harmonic in range(1,harmo_num+1):
        
        # update the opration_flag
        ops_num +=1
                
        # use getInfo fuction to get the unpack values 
        unpacked_value = ee.Dictionary(point_with_residule[year][harmonic]).getInfo()


        # unpack the inner dict into [pt_id:value]
        for pt_id,v in unpacked_value.items():
            residules = v[1][-3:]

            # unpack the residules and gave it a corresponding index name
            for index,residule in zip(['NDVI','NDBI','EVI'],residules):
                Residule.update({(year,harmonic,pt_id,index):[residule]})
                
        # print out the process
        print(f'Compute and unpack year_{year}_Harmonic_{harmonic} successful!  ==>{ops_num}/{ops_total}') 
 


In [ ]:
Residule